<a href="https://colab.research.google.com/github/noea599/Test/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Cleaning and Preparation

This involves the necessary steps and processing to get both the train and test datasets in an acceptable form for the machine learnning algorithm:

* Creating validation data sets
* Selecting interested logs
* Resolving missing values
* Encoding categorical variables
* Data augmentation

In [ ]:
import numpy as np

import random
import pandas as pd
import xgboost as xgb 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
Booster.save_model()

In [ ]:
testdata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CSV_test.csv', sep=';')
traindata = pd.read_csv('/content/drive/MyDrive/CSV_train.csv', sep=';')

testdata

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,SP,BS,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO
0,15/9-14,480.628001,423244.50000,6461862.5,-455.624420,NORDLAND GP.,NaN,19.2031,NaN,1.613886,...,35.525719,NaN,96.461990,NaN,NaN,-0.538873,0.130611,NaN,NaN,NaN
1,15/9-14,480.780001,423244.50000,6461862.5,-455.776428,NORDLAND GP.,NaN,19.2031,NaN,1.574376,...,36.158520,NaN,96.454399,NaN,NaN,-0.539232,0.130611,NaN,NaN,NaN
2,15/9-14,480.932001,423244.50000,6461862.5,-455.928436,NORDLAND GP.,NaN,19.2031,NaN,1.436627,...,36.873703,NaN,96.446686,NaN,NaN,-0.540830,0.130611,NaN,NaN,NaN
3,15/9-14,481.084001,423244.50000,6461862.5,-456.080444,NORDLAND GP.,NaN,19.2031,NaN,1.276094,...,37.304054,NaN,161.170166,NaN,NaN,-0.543943,0.130611,NaN,NaN,NaN
4,15/9-14,481.236001,423244.53125,6461862.5,-456.232422,NORDLAND GP.,NaN,19.2031,NaN,1.204704,...,37.864922,NaN,172.489120,NaN,NaN,-0.542104,0.130611,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136781,35/9-8,3224.389600,536225.93750,6794880.5,-3199.876465,BAAT GP.,Rannoch Fm.,8.4978,NaN,6.231942,...,NaN,8.5,26.615782,118.669212,NaN,0.063478,NaN,2.618309,33.523922,NaN
136782,35/9-8,3224.541600,536225.93750,6794880.5,-3200.028320,BAAT GP.,Rannoch Fm.,8.4978,NaN,6.038777,...,NaN,8.5,25.647141,118.468925,NaN,0.056791,NaN,2.620221,32.643795,NaN
136783,35/9-8,3224.693600,536225.93750,6794880.5,-3200.180176,BAAT GP.,Rannoch Fm.,8.4978,NaN,5.503983,...,NaN,8.5,23.929407,118.163177,NaN,0.002499,NaN,2.629171,31.763380,NaN
136784,35/9-8,3224.845600,536225.93750,6794880.5,-3200.332031,BAAT GP.,Rannoch Fm.,8.4978,NaN,4.895551,...,NaN,8.5,22.737293,117.655937,NaN,0.003363,NaN,2.521121,30.884350,NaN


In [ ]:
A = np.load('/content/drive/MyDrive/penalty_matrix.npy')   # penalty matrix used for scoring

In [ ]:
A

array([[0.   , 2.   , 3.5  , 3.   , 3.75 , 3.5  , 3.5  , 4.   , 4.   ,
        2.5  , 3.875, 3.25 ],
       [2.   , 0.   , 2.375, 2.75 , 4.   , 3.75 , 3.75 , 3.875, 4.   ,
        3.   , 3.75 , 3.   ],
       [3.5  , 2.375, 0.   , 2.   , 3.5  , 3.5  , 3.75 , 4.   , 4.   ,
        2.75 , 3.25 , 3.   ],
       [3.   , 2.75 , 2.   , 0.   , 2.5  , 2.   , 2.25 , 4.   , 4.   ,
        3.375, 3.75 , 3.25 ],
       [3.75 , 4.   , 3.5  , 2.5  , 0.   , 2.625, 2.875, 3.75 , 3.25 ,
        3.   , 4.   , 3.625],
       [3.5  , 3.75 , 3.5  , 2.   , 2.625, 0.   , 1.375, 4.   , 3.75 ,
        3.5  , 4.   , 3.625],
       [3.5  , 3.75 , 3.75 , 2.25 , 2.875, 1.375, 0.   , 4.   , 3.75 ,
        3.125, 4.   , 3.75 ],
       [4.   , 3.875, 4.   , 4.   , 3.75 , 4.   , 4.   , 0.   , 2.75 ,
        3.75 , 3.75 , 4.   ],
       [4.   , 4.   , 4.   , 4.   , 3.25 , 3.75 , 3.75 , 2.75 , 0.   ,
        4.   , 4.   , 3.875],
       [2.5  , 3.   , 2.75 , 3.375, 3.   , 3.5  , 3.125, 3.75 , 4.   ,
        0.   , 2.5  

Scoring matrix for petrophysical interpretation

![Penalty.png](images/penalty_matrix.jpg)

In [ ]:
def score(y_true, y_pred):

    '''
    custom metric used for evaluation
    args:
      y_true: actual prediction
      y_pred: predictions made
    '''

    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

### Creating Validation Sets

Validation sets are created to properly evaluate changes made on the machine learning model. This is important to prevent overfitting the open test data since the blind test is used as the final determiner. So it is important to build ML models that will generalise better on unseen wells. Having no idea how the blind wells would come (geospatial distribution, logs presence), there was no specific guide in selecting train wells, so wells were randomly selected from the train to create two validation sets (each comprising of 10 wells). 

In [ ]:
train_wells = traindata.WELL.unique()

In [ ]:
print(f'Initial total number of train wells: {len(train_wells)}')

Initial total number of train wells: 98


In [ ]:
np.random.seed(12)

In [ ]:
valid1 = random.sample(list(train_wells), 10)   #randomly sample 10 well names from train data

In [ ]:
valid1

['30/3-5 S',
 '35/11-10',
 '31/3-3',
 '33/9-17',
 '34/8-7 R',
 '25/2-14',
 '34/7-13',
 '25/8-7',
 '34/7-21',
 '25/6-2']

QC to remove valid1 wells from train wells to prevent having same well(s) in the second validation data

In [ ]:
train_wells = [well for well in train_wells if not well in valid1]
print(f'Number of wells left: {len(train_wells)}')

Number of wells left: 88


In [ ]:
valid2 = random.sample(list(train_wells), 10)

train_wells = [well for well in train_wells if not well in valid2]
print(f'Number of wells left: {len(train_wells)}')

Number of wells left: 78


In [ ]:
print(len(valid1), len(valid2))

10 10


In [ ]:
validation_wells = set(valid1 + valid2)
print(len(validation_wells))

20


Let's proceed to getting the validation data from the train data set and dropping them to prevent any form of data leakage.

In [ ]:
def create_validation_set(train, wells):
    
    '''
    Function to validation sets from the full train data using well names
    '''
    
    validation = pd.DataFrame(columns=list(train.columns))
    
    for well in wells:
        welldata = train.loc[train.WELL == well]
        validation = pd.concat((welldata, validation))
        
    return validation

In [ ]:
# using function to get data for validation wells

validation1 = create_validation_set(traindata, valid1)
validation2 = create_validation_set(traindata, valid2)

In [ ]:
validation2

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
142336,16/4-1,760.889604,449950.6875,6500259.0,-735.806519,NORDLAND GP.,NaN,16.067059,0.987789,0.985453,...,0.484298,NaN,1.317059,0.001534,0.140197,NaN,NaN,NaN,30000,1.0
142337,16/4-1,761.041604,449950.6875,6500259.0,-735.958557,NORDLAND GP.,Utsira Fm.,16.499599,0.864718,0.775014,...,0.468800,NaN,1.749599,-0.003222,0.140197,NaN,NaN,NaN,30000,1.0
142338,16/4-1,761.193604,449950.6875,6500259.0,-736.110535,NORDLAND GP.,Utsira Fm.,17.001528,0.762343,0.656081,...,0.468800,NaN,2.251528,-0.002393,0.140197,NaN,NaN,NaN,30000,1.0
142339,16/4-1,761.345604,449950.6875,6500259.0,-736.262512,NORDLAND GP.,Utsira Fm.,17.524185,0.689617,0.619384,...,0.468800,NaN,2.774185,0.002004,0.140197,NaN,NaN,NaN,30000,1.0
142340,16/4-1,761.497604,449950.6875,6500259.0,-736.414551,NORDLAND GP.,Utsira Fm.,18.162369,0.646507,0.620586,...,0.438160,NaN,3.412369,0.003000,0.140197,NaN,NaN,NaN,30000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070938,35/12-1,3001.630815,551806.9375,6783866.0,-2973.820801,DUNLIN GP.,Cook Fm.,12.777927,1.525746,2.589892,...,NaN,NaN,NaN,0.019719,NaN,NaN,NaN,1.860739,30000,2.0
1070939,35/12-1,3001.782815,551806.9375,6783866.0,-2973.972656,DUNLIN GP.,Cook Fm.,12.770145,1.588097,3.013698,...,NaN,NaN,NaN,0.018917,NaN,NaN,NaN,1.572933,30000,2.0
1070940,35/12-1,3001.934815,551806.9375,6783866.0,-2974.124756,DUNLIN GP.,Cook Fm.,12.768833,2.341989,3.333388,...,NaN,NaN,NaN,0.017262,NaN,NaN,NaN,1.570367,30000,2.0
1070941,35/12-1,3002.086815,551806.9375,6783866.0,-2974.276611,DUNLIN GP.,Cook Fm.,12.773399,3.759902,3.378008,...,NaN,NaN,NaN,0.011838,NaN,NaN,NaN,2.964980,30000,2.0


In [ ]:
# total validation data

validation = pd.concat((validation1, validation2))

In [ ]:
validation.shape, validation1.shape, validation2.shape

((240012, 29), (91102, 29), (148910, 29))

In [ ]:
# dropping validation data from train data

new_train = pd.concat([traindata, validation, validation]).drop_duplicates(keep=False)
print(f'Previous train data shape: {traindata.shape}')
print(f'New train data shape: {new_train.shape}')

Previous train data shape: (1170511, 29)
New train data shape: (930499, 29)


In [ ]:
# QC to ensure there are no data leakage

previous_rows = traindata.shape[0]
new_train_rows = new_train.shape[0]
validation_rows = validation.shape[0]

print(f'Number of previous train data rows: {previous_rows}')
print(f'Validation + new train rows: {validation_rows+new_train_rows}')

Number of previous train data rows: 1170511
Validation + new train rows: 1170511


In [ ]:
# to confirm we still have all samples of the labels in the train data set


new_train.FORCE_2020_LITHOFACIES_LITHOLOGY.value_counts()

65000    572475
30000    137837
65030    120095
70000     44773
80000     26187
99000     12643
70032      8015
88000      3919
90000      2989
74000      1246
86000       320
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, dtype: int64

Now we can proceed to other preparation procedures.

Logs were selected based on user's desire to use them for training. The confidence logs was dropped as this was absent in the test logs as the ML models need the same set of wells used for training in making predictions on the test wells. Other absent logs and logs with low percentage of values from the combined test data are also dropped. This has previously been visualized in the EDA notebook. A cut off of 30% may be selected as criteria for dropping the logs. Let's take a look at that!

In [ ]:
print(f'Percentage of values in test logs:')
100 - testdata.isna().sum()/testdata.shape[0]*100

Percentage of values in test logs:


WELL         100.000000
DEPTH_MD     100.000000
X_LOC         99.956867
Y_LOC         99.956867
Z_LOC         99.956867
GROUP        100.000000
FORMATION     94.828418
CALI          95.873116
RSHA          28.582603
RMED          99.570863
RDEP          99.956867
RHOB          87.601070
GR           100.000000
SGR            0.000000
NPHI          76.062609
PEF           82.978521
DTC           99.398330
SP            48.708932
BS            48.955302
ROP           49.943708
DTS           31.596801
DCAL           9.880397
DRHO          81.555130
MUDWEIGHT     14.818037
RMIC           8.272776
ROPA          40.786338
RXO           21.820947
dtype: float64

For better and faster processing, the train, validation and test data sets will be concatenated and processed together as we need these data sets to be in the same formats to get good predictions out of the ML model. But let's have it in mind that the RSHA, SGR, DCAL, MUDWEIGHT, RMIC and RXO will be dropped from the wells. Let's proceed.

Let's extract the data sets indices that will be used for splitting the features and targets into their respective datasets after prepration is complete. We will also be extracting the train target

In [ ]:
ntrain = new_train.shape[0]
ntest = testdata.shape[0]
nvalid1 = validation1.shape[0]
nvalid2 = validation2.shape[0]
nvalid3 = validation.shape[0]

df = pd.concat((new_train, testdata, validation1, validation2, validation)).reset_index(drop=True)


So the combined dataframe for preparation

![Picture.png](images/Picture3.png)

In [ ]:
df.shape

(1547309, 29)

The procedure below is used to extract data needed for the augmentation procedure to be performed after every other preparation has been done

In [ ]:
#making a copy of the dataframes

train = new_train.copy()
test = testdata.copy()
valid1 = validation1.copy()
valid2 = validation2.copy()
valid = validation.copy()

In [ ]:
# extracting the data sets well names and depth values needed for augmentation

train_well = train.WELL.values
train_depth = train.DEPTH_MD.values

test_well = test.WELL.values
test_depth = test.DEPTH_MD.values
 
valid1_well = valid1.WELL.values
valid1_depth = valid1.DEPTH_MD.values
 
valid2_well = valid2.WELL.values
valid2_depth = valid2.DEPTH_MD.values
 
valid_well = valid.WELL.values
valid_depth = valid.DEPTH_MD.values

In [ ]:
valid1_depth

array([1924.616    , 1924.768    , 1924.92     , ..., 4560.7295862,
       4560.8815862, 4561.0335862])

Now let's extract the data sets labels and prepare them for training and validation performance check.

In [ ]:
lithology = train['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid1_lithology = valid1['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid2_lithology = valid2['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid_lithology = valid['FORCE_2020_LITHOFACIES_LITHOLOGY']
 
lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}
 
lithology = lithology.map(lithology_numbers)
valid1_lithology = valid1_lithology.map(lithology_numbers)
valid2_lithology = valid2_lithology.map(lithology_numbers)
valid_lithology = valid_lithology.map(lithology_numbers)


In [ ]:
valid1_lithology

370045    2
370046    2
370047    2
370048    2
370049    2
         ..
517753    0
517754    0
517755    0
517756    0
517757    0
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, Length: 91102, dtype: int64

### Data Encoding

The categorical logs/columns in the data set need to be encoded for use by the ML algorithm. From the data visualization, we saw the high dimensionality of the logs (especially the FORMATION log with 69 distinct values), so label encoding will be applied instead of one hot encoding these features to prevent high dimensionality of the data.

In [ ]:
df.shape

(1547309, 29)

In [ ]:
df['GROUP_encoded'] = df['GROUP'].astype('category')
df['GROUP_encoded'] = df['GROUP_encoded'].cat.codes 

df['FORMATION_encoded'] = df['FORMATION'].astype('category')
df['FORMATION_encoded'] = df['FORMATION_encoded'].cat.codes

df['WELL_encoded'] = df['WELL'].astype('category')
df['WELL_encoded'] = df['WELL_encoded'].cat.codes

print(f'shape of dataframe after label encoding columns {df.shape}')

shape of dataframe after label encoding columns (1547309, 32)


In [ ]:
df['GROUP_encoded']

0          6
1          6
2          6
3          6
4          6
          ..
1547304    3
1547305    3
1547306    3
1547307    3
1547308    3
Name: GROUP_encoded, Length: 1547309, dtype: int8

In [ ]:
df.columns

Index(['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP', 'FORMATION',
       'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR', 'NPHI', 'PEF',
       'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC',
       'ROPA', 'RXO', 'FORCE_2020_LITHOFACIES_LITHOLOGY',
       'FORCE_2020_LITHOFACIES_CONFIDENCE', 'GROUP_encoded',
       'FORMATION_encoded', 'WELL_encoded'],
      dtype='object')

In [ ]:
df.GROUP_encoded.value_counts()

 5     373013
 9     330541
 7     171654
 12    167188
 3     162459
 6     140958
 2      68076
 0      52750
 11     33870
 13     18840
 4      16781
 8       5301
 1       3395
-1       1278
 10      1205
Name: GROUP_encoded, dtype: int64

In [ ]:
# dropping the previous columns after encoding

df = df.drop(['WELL', 'GROUP', 'FORMATION'], axis=1)

#droppng uncessary columns

df = df.drop([ "RSHA", "SGR", "DCAL", "MUDWEIGHT", "RMIC" , "RXO","FORCE_2020_LITHOFACIES_CONFIDENCE"],axis=1)


In [ ]:
df.isna().sum()/df.shape[0]*100

DEPTH_MD                             0.000000
X_LOC                                1.199696
Y_LOC                                1.199696
Z_LOC                                1.199696
CALI                                 6.984319
RMED                                 3.409662
RDEP                                 1.215207
RHOB                                13.628952
GR                                   0.000000
NPHI                                33.687906
PEF                                 40.593508
DTC                                  6.191071
SP                                  28.392390
BS                                  40.406603
ROP                                 54.315718
DTS                                 84.294023
DRHO                                15.410044
ROPA                                81.171505
FORCE_2020_LITHOFACIES_LITHOLOGY     8.840251
GROUP_encoded                        0.000000
FORMATION_encoded                    0.000000
WELL_encoded                      

### Filling Missing Values


Some fractions of missing values still exist in present logs, how do we resolve that? While we can use a mean of values in a window to solve this, backward or forward fill, we could also decide to fill up all missing values with a distinct value different from other values. This way, the ML algorithm used (in this case a gradient tree algorithm) can differentiate this better. From validation, this improved result better. -9999 is used, and since this is a classification problem as opposed to a regression where we predict actual lithology values, outlier effect is not observed in the output.

In [ ]:
df = df.fillna(-9999)

In [ ]:
df.isna().sum()/df.shape[0]*100

DEPTH_MD                            0.0
X_LOC                               0.0
Y_LOC                               0.0
Z_LOC                               0.0
CALI                                0.0
RMED                                0.0
RDEP                                0.0
RHOB                                0.0
GR                                  0.0
NPHI                                0.0
PEF                                 0.0
DTC                                 0.0
SP                                  0.0
BS                                  0.0
ROP                                 0.0
DTS                                 0.0
DRHO                                0.0
ROPA                                0.0
FORCE_2020_LITHOFACIES_LITHOLOGY    0.0
GROUP_encoded                       0.0
FORMATION_encoded                   0.0
WELL_encoded                        0.0
dtype: float64

Now that we've completed the majority of the preparation, let's split back our concatenated dataframe into their validation sets, train and test set

In [ ]:
data = df.copy()   #making a copy of the preparaed dataframe to work with
data.shape

(1547309, 22)

Remember the shape of the concatenated dataframe;

![Picture.png](images/Picture3.png)

using the data sets indices will be used for slicing out their corresponding features

In [ ]:
train = data[:ntrain].copy()
train.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
        
test = data[ntrain:(ntest+ntrain)].copy()
test.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
test = test.reset_index(drop=True)

valid1 = data[(ntest+ntrain):(ntest+ntrain+nvalid1)].copy()
valid1.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
valid1 = valid1.reset_index(drop=True)

valid2 = data[(ntest+ntrain+nvalid1):(ntest+ntrain+nvalid1+nvalid2)].copy()
valid2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
valid2 = valid2.reset_index(drop=True)

valid = data[(ntest+ntrain+nvalid1+nvalid2):].copy()
valid.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
valid = valid.reset_index(drop=True)

In [ ]:
# checking shapes of sliced data sets for QC






train.shape, test.shape, valid1.shape, valid2.shape, valid.shape

((930499, 21), (136786, 21), (91102, 21), (148910, 21), (240012, 21))

In [ ]:
valid1.columns

Index(['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RMED', 'RDEP', 'RHOB',
       'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO', 'ROPA',
       'GROUP_encoded', 'FORMATION_encoded', 'WELL_encoded'],
      dtype='object')

In [ ]:
type(valid1)

pandas.core.frame.DataFrame

### Data Augmentation

The data augmentation technique is extracted from the ISPL team code for the 2016 SEG ML competition : https://github.com/seg/2016-ml-contest/tree/master/ispl . The technique was based on the assumption that "facies do not abrutly change from a given depth layer to the next one". This was implemented by aggregating features at neighbouring depths and computing the feature spatial gradient.

In [ ]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]
 
    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))
 
    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row
 
    return X_aug
 
# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad
 
# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    return X_aug

In [ ]:
print(f'Shape of datasets before augmentation {train.shape, test.shape, valid1.shape, valid2.shape, valid.shape}')

aug_train = augment_features(train.values, train_well, train_depth)
aug_test = augment_features(test.values, test_well, test_depth)
aug_valid1 = augment_features(valid1.values, valid1_well, valid1_depth)
aug_valid2 = augment_features(valid2.values, valid2_well, valid2_depth)
aug_valid = augment_features(valid.values, valid_well, valid_depth)

print(f'Shape of datasets after augmentation {aug_train.shape, aug_test.shape, aug_valid1.shape, aug_valid2.shape, aug_valid.shape}')

Shape of datasets before augmentation ((930499, 21), (136786, 21), (91102, 21), (148910, 21), (240012, 21))
Shape of datasets after augmentation ((930499, 84), (136786, 84), (91102, 84), (148910, 84), (240012, 84))


In [ ]:
pd.DataFrame(aug_train).head(10)

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,0.000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,-3.471776,-4.716108,0.0,0.000000,0.0,0.031179,0.0,0.0,0.0,0.0
1,494.528,437641.96875,6470972.5,-469.501831,19.480835,1.611410,1.798681,1.884186,80.200851,-9999.0,...,-2.827996,0.137015,0.0,0.941753,0.0,-0.026689,0.0,0.0,0.0,0.0
2,494.680,437641.96875,6470972.5,-469.653809,19.468800,1.618070,1.795641,1.889794,79.262886,-9999.0,...,-0.159113,-0.807034,0.0,34.115842,0.0,-0.079409,0.0,0.0,0.0,0.0
3,494.832,437641.96875,6470972.5,-469.805786,19.468800,1.626459,1.800733,1.896523,74.821999,-9999.0,...,-0.138735,2.042043,0.0,115.253950,0.0,-0.076305,0.0,0.0,0.0,0.0
4,494.984,437641.96875,6470972.5,-469.957794,19.459282,1.621594,1.801517,1.891913,72.878922,-9999.0,...,0.137831,-1.136843,0.0,117.089773,0.0,-0.024252,0.0,0.0,0.0,0.0
5,495.136,437641.96875,6470972.5,-470.109772,19.453100,1.602679,1.795299,1.880034,71.729141,-9999.0,...,7.244010,-3.615053,0.0,6.043033,0.0,0.021260,0.0,0.0,0.0,0.0
6,495.288,437641.96875,6470972.5,-470.261780,19.453100,1.585567,1.804719,1.879687,72.014420,-9999.0,...,6.342336,1.647209,0.0,0.000000,0.0,-0.024150,0.0,0.0,0.0,0.0
7,495.440,437641.96875,6470972.5,-470.413788,19.462496,1.576569,1.805498,1.878731,72.588089,-9999.0,...,-4.206005,-9.662001,0.0,-1.981283,0.0,-0.081083,0.0,0.0,0.0,0.0
8,495.592,437641.96875,6470972.5,-470.565796,19.468800,1.587011,1.808367,1.867837,71.283051,-9999.0,...,-6.524638,9.844629,0.0,-51.167990,0.0,-0.049004,0.0,0.0,0.0,0.0
9,495.744,437641.96875,6470972.5,-470.717773,19.468800,1.613674,1.815813,1.847233,69.721436,-9999.0,...,-6.309007,2.492102,0.0,-137.461311,0.0,0.065673,0.0,0.0,0.0,0.0


In [ ]:
train.head(10)

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RMED,RDEP,RHOB,GR,NPHI,...,DTC,SP,BS,ROP,DTS,DRHO,ROPA,GROUP_encoded,FORMATION_encoded,WELL_encoded
0,494.528,437641.96875,6470972.5,-469.501831,19.480835,1.611410,1.798681,1.884186,80.200851,-9999.0,...,161.131180,24.612379,-9999.0,34.636410,-9999.0,-0.574928,-9999.0,6,-1,0
1,494.680,437641.96875,6470972.5,-469.653809,19.468800,1.618070,1.795641,1.889794,79.262886,-9999.0,...,160.603470,23.895531,-9999.0,34.636410,-9999.0,-0.570188,-9999.0,6,-1,0
2,494.832,437641.96875,6470972.5,-469.805786,19.468800,1.626459,1.800733,1.896523,74.821999,-9999.0,...,160.173615,23.916357,-9999.0,34.779556,-9999.0,-0.574245,-9999.0,6,-1,0
3,494.984,437641.96875,6470972.5,-469.957794,19.459282,1.621594,1.801517,1.891913,72.878922,-9999.0,...,160.149429,23.793688,-9999.0,39.965164,-9999.0,-0.586315,-9999.0,6,-1,0
4,495.136,437641.96875,6470972.5,-470.109772,19.453100,1.602679,1.795299,1.880034,71.729141,-9999.0,...,160.128342,24.104078,-9999.0,57.483765,-9999.0,-0.597914,-9999.0,6,-1,0
5,495.288,437641.96875,6470972.5,-470.261780,19.453100,1.585567,1.804719,1.879687,72.014420,-9999.0,...,160.149292,23.931278,-9999.0,75.281410,-9999.0,-0.601600,-9999.0,6,-1,0
6,495.440,437641.96875,6470972.5,-470.413788,19.462496,1.576569,1.805498,1.878731,72.588089,-9999.0,...,161.250381,23.381790,-9999.0,76.199951,-9999.0,-0.598369,-9999.0,6,-1,0
7,495.592,437641.96875,6470972.5,-470.565796,19.468800,1.587011,1.808367,1.867837,71.283051,-9999.0,...,162.214416,23.632166,-9999.0,76.199951,-9999.0,-0.602039,-9999.0,6,-1,0
8,495.744,437641.96875,6470972.5,-470.717773,19.468800,1.613674,1.815813,1.847233,69.721436,-9999.0,...,161.575104,22.163542,-9999.0,75.898796,-9999.0,-0.614364,-9999.0,6,-1,0
9,495.896,437641.96875,6470972.5,-470.869781,19.468800,1.634622,1.813916,1.836309,66.677727,-9999.0,...,160.583359,23.659925,-9999.0,68.121262,-9999.0,-0.621813,-9999.0,6,-1,0


### Model Training

The choice of algorithm for this tutorial workflow is xgboost. Why? Performance on previously done validation was better, and also at a faster compute speed than catboost. Random forest is also a great algorithm to try. Let's implement our xgboost tree. This will be done in a 10 fold cross validation technique. This is done to get a better performance and a confident result that is not due to randomness. We will be using StratifiedKFold function from sklearn. Let's look at that.

In [ ]:
def show_evaluation(pred, true):

  '''

  function to show model performance and evaluation
  args:
    pred: predicted value(a list)
    true: actual values (a list)

  prints the custom metric performance, accuracy and F1 score of predictions

  '''

  print(f'Default score: {score(true.values, pred)}')
  print(f'Accuracy is: {accuracy_score(true, pred)}')
  print(f'F1 is: {f1_score(pred, true.values, average="weighted")}')

In [ ]:
# initializing the xgboost model

model = xgb.XGBClassifier(n_estimators=100, max_depth=10, booster='gbtree',
                          objective='softprob', learning_rate=0.1, random_state=0,
                          subsample=0.9, colsample_bytree=0.9, tree_method='hist',
                          eval_metric='mlogloss', reg_lambda=1500, n_jobs=-1)

In [ ]:
split = 10

kf = StratifiedKFold(n_splits=split, shuffle=True)

In [ ]:
# initializing the prediction probabilities arrays

test_pred = np.zeros((len(test), 12))
valid1_pred = np.zeros((len(valid1), 12))
valid2_pred = np.zeros((len(valid2), 12))
valid_pred = np.zeros((len(valid), 12))

In [ ]:
test_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#implementing the CV Loop

i = 1
for (train_index, test_index) in kf.split(train, lithology):
    # print(train_index.shape, test_index.shape)
    # print(train_index, test_index)
    X_train,X_test = train.iloc[train_index], train.iloc[test_index]
    Y_train,Y_test = lithology.iloc[train_index], lithology.iloc[test_index]
       
    model.fit(X_train, Y_train, early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=1)
    
    prediction1 = model.predict(valid1)
    prediction2 = model.predict(valid2)
    prediction = model.predict(valid)
    
    print(show_evaluation(prediction1, valid1_lithology))
    print(show_evaluation(prediction2, valid2_lithology))
    print(show_evaluation(prediction, valid_lithology))
 
    print(f'----------------------- FOLD {i} ---------------------')
    i+=1
    
    # valid1_pred += model.predict_proba(valid1)
    # valid2_pred += model.predict_proba(valid2)
    # valid_pred += model.predict_proba(valid)

[0]	validation_0-mlogloss:2.10601
Will train until validation_0-mlogloss hasn't improved in 100 rounds.
[1]	validation_0-mlogloss:1.9025
[2]	validation_0-mlogloss:1.73938
[3]	validation_0-mlogloss:1.60507
[4]	validation_0-mlogloss:1.49289
[5]	validation_0-mlogloss:1.39522
[6]	validation_0-mlogloss:1.30849
[7]	validation_0-mlogloss:1.23372
[8]	validation_0-mlogloss:1.16621
[9]	validation_0-mlogloss:1.10591
[10]	validation_0-mlogloss:1.05123
[11]	validation_0-mlogloss:1.00321
[12]	validation_0-mlogloss:0.95945
[13]	validation_0-mlogloss:0.919779
[14]	validation_0-mlogloss:0.882924
[15]	validation_0-mlogloss:0.849119
[16]	validation_0-mlogloss:0.818108
[17]	validation_0-mlogloss:0.789362
[18]	validation_0-mlogloss:0.763044
[19]	validation_0-mlogloss:0.738732
[20]	validation_0-mlogloss:0.717345
[21]	validation_0-mlogloss:0.696893
[22]	validation_0-mlogloss:0.678208
[23]	validation_0-mlogloss:0.659999
[24]	validation_0-mlogloss:0.643606
[25]	validation_0-mlogloss:0.628023
[26]	validation_0-

In [ ]:
valid1.columns
model.save_model("model_lithoPred1.json")

In [ ]:
import pickle
filename = 'model_lithoPred.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
result = model.predict(valid1)


In [ ]:
valid1.columns

Index(['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RMED', 'RDEP', 'RHOB',
       'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO', 'ROPA',
       'FORCE_2020_LITHOFACIES_CONFIDENCE', 'GROUP_encoded',
       'FORMATION_encoded', 'WELL_encoded'],
      dtype='object')

In [ ]:
result_df = pd.DataFrame(result)
result_df.columns = ["pred"]
result_df

,pred
0,2
1,2
2,2
3,2
4,2
...,...
122911,2
122912,2
122913,2
122914,2


In [ ]:
# finding the probabilities average and converting the numpy array to a dataframe
valid1_pred = model.predict_proba(valid1)
valid2_pred  =model.predict_proba(valid2)
valid_pred= model.predict_proba(valid)

valid1_pred = pd.DataFrame(valid1_pred/split)
valid2_pred = pd.DataFrame(valid2_pred/split)
valid_pred = pd.DataFrame(valid_pred/split)

In [ ]:
# extracting the index position with the highest probability as the lithology classp

valid1_pred = valid1_pred.idxmax(axis=1)
valid2_pred = valid2_pred.idxmax(axis=1)
valid_pred = valid_pred.idxmax(axis=1)

Evaluating the final predictions from the CV and max probability indexing

In [ ]:
show_evaluation(valid1_pred, valid1_lithology)

In [ ]:
show_evaluation(valid2_pred, valid2_lithology)

In [ ]:
show_evaluation(valid_pred, valid_lithology)

In [ ]:
valid1_lithology_predict =model.predict(valid1)